## Kafka Ingestion

### Required Imports + Subscribing to topic

In [1]:
from confluent_kafka import Consumer
import uuid
import os
from dotenv import load_dotenv
import snowflake.connector
import os
load_dotenv()

KAFKA_SERVER = os.getenv('KAFKA_SERVER')
KAFKA_USERNAME=os.getenv('KAFKA_USERNAME')
KAFKA_PASSWORD=os.getenv('KAFKA_PASSWORD')
KAFKA_TOPIC_NAME = os.getenv('KAFKA_TOPIC_NAME')

values = []

# Consumer with `confluent_kafka`
c = Consumer({
    'bootstrap.servers': KAFKA_SERVER,
    'group.id': f'deleton' +str(uuid.uuid1()),
    'security.protocol': 'SASL_SSL',
    'sasl.mechanisms': 'PLAIN',
    'sasl.username': KAFKA_USERNAME,
    'sasl.password': KAFKA_PASSWORD,
    'session.timeout.ms': 6000,
    'heartbeat.interval.ms': 1000,
    'fetch.wait.max.ms': 6000,
    'auto.offset.reset': 'latest',
    'enable.auto.commit': 'false',
    'max.poll.interval.ms': '86400000',
    'topic.metadata.refresh.interval.ms': "-1",
    "client.id": 'id-002-005',
})

c.subscribe([KAFKA_TOPIC_NAME])
"""Kafka consumer ready and consumes messages"""

/opt/homebrew/lib/python3.9/site-packages/snowflake/connector/options.py:96: UserWarning: You have an incompatible version of 'pyarrow' installed (9.0.0), please install a version that adheres to: 'pyarrow<8.1.0,>=8.0.0; extra == "pandas"'
  warn_incompatible_dep(


'Kafka consumer ready and consumes messages'

### Testing Kafka Messages

In [35]:
import json
import re
i = 0
no_system_log = True
while no_system_log:
    kafka_message = c.poll(0.5) #poll all messages

    if kafka_message is not None: #exclude none values
        log = kafka_message.value().decode('utf-8')

        if 'INFO' in log: #only check for strings with INFO
            values = json.loads(log)
            new_log = values.get('log')
            print(new_log) #Get value inside dict

            if 'Ride' in new_log: #process strings with Ride info
                new_log = new_log.split(' mendoza v9: [INFO]: Ride - ')
                timestamp = new_log[0]
                log_values = re.findall(r'\d+.?\d+',new_log[1])

                duration=log_values[0]
                resistance=log_values[1]

           
            if 'Telemetry' in new_log:
                new_log = new_log.split(' mendoza v9: [INFO]: Telemetry - ')
                timestamp = new_log[0]
                log_values = re.findall(r'\d+.?\d+',new_log[1])
                heart_rate = log_values[0]
                rpm = log_values[1]
                power= log_values[2]
                
        if 'SYSTEM' in log:
            system_log = json.loads(system_log).get('log')
            split_log = system_log.split(' mendoza v9: [SYSTEM] data = ')
            timestamp = split_log[0]
            dictionary_string = split_log[1][:-1]
            user_dictionary = json.loads(dictionary_string)


    i += 1

2022-10-05 07:31:23.565975 mendoza v9: [INFO]: Telemetry - hrt = 160; rpm = 49; power = 13.448875699999999

2022-10-05 07:31:23.565975 ['160', '49', '13.448875699999999']
2022-10-05 07:31:24.066664 mendoza v9: [INFO]: Ride - duration = 147.0; resistance = 30

2022-10-05 07:31:24.567348 mendoza v9: [INFO]: Telemetry - hrt = 160; rpm = 51; power = 14.31786487

2022-10-05 07:31:24.567348 ['160', '51', '14.31786487']
2022-10-05 07:31:25.068017 mendoza v9: [INFO]: Ride - duration = 148.0; resistance = 30

2022-10-05 07:31:25.568892 mendoza v9: [INFO]: Telemetry - hrt = 160; rpm = 50; power = 13.88151165

2022-10-05 07:31:25.568892 ['160', '50', '13.88151165']
2022-10-05 07:31:26.069563 mendoza v9: [INFO]: Ride - duration = 149.0; resistance = 30

2022-10-05 07:31:26.570247 mendoza v9: [INFO]: Telemetry - hrt = 160; rpm = 49; power = 13.448875699999999

2022-10-05 07:31:26.570247 ['160', '49', '13.448875699999999']
2022-10-05 07:31:27.070918 mendoza v9: [INFO]: Ride - duration = 150.0; resis

KeyboardInterrupt: 

In [32]:
system_log = json.loads(system_log).get('log')
split_log = system_log.split(' mendoza v9: [SYSTEM] data = ')
timestamp = split_log[0]
dictionary_string = split_log[1][:-1]
user_dictionary = json.loads(dictionary_string)
user_dictionary

{'user_id': 4558,
 'name': 'Brian Norris-King',
 'gender': 'male',
 'address': '08 Linda spring,Smithland,ZE3M 0ZH',
 'date_of_birth': 351820800000,
 'email_address': 'brian.norris-king5@hotmail.com',
 'height_cm': 169,
 'weight_kg': 59,
 'account_create_date': 1638316800000,
 'bike_serial': 'SN0000',
 'original_source': 'offline'}

%4|1664955026.033|SESSTMOUT|id-002-005#consumer-1| [thrd:main]: Consumer group session timed out (in join-state steady) after 167867 ms without a successful response from the group coordinator (broker 6, last error was Success): revoking assignment and rejoining group
%5|1664955026.033|REQTMOUT|id-002-005#consumer-1| [thrd:sasl_ssl://b13-pkc-l6wr6.europe-west2.gcp.confluent.cloud:9092/]: sasl_ssl://b13-pkc-l6wr6.europe-west2.gcp.confluent.cloud:9092/13: Timed out FetchRequest in flight (after 167236ms, timeout #0)
%4|1664955026.033|REQTMOUT|id-002-005#consumer-1| [thrd:sasl_ssl://b13-pkc-l6wr6.europe-west2.gcp.confluent.cloud:9092/]: sasl_ssl://b13-pkc-l6wr6.europe-west2.gcp.confluent.cloud:9092/13: Timed out 1 in-flight, 0 retry-queued, 0 out-queue, 0 partially-sent requests
%3|1664955026.033|FAIL|id-002-005#consumer-1| [thrd:sasl_ssl://b13-pkc-l6wr6.europe-west2.gcp.confluent.cloud:9092/]: sasl_ssl://b13-pkc-l6wr6.europe-west2.gcp.confluent.cloud:9092/13: 1 request(s) timed out: disc

### TODO: 

* Read kafka data into a dataframe (Pandas or PySpark?)
* Write data into Snowflake

In [ ]:
USER = os.environ.get('USER')
ACCOUNT = os.environ.get('ACCOUNT')
PASSWORD = os.environ.get('PASSWORD')
WAREHOUSE= os.environ.get('WAREHOUSE')
DATABASE= os.environ.get('DATABASE')
SCHEMA= os.environ.get('SCHEMA')

In [ ]:
conn = snowflake.connector.connect(
    user=USER,
    password=PASSWORD,
    account=ACCOUNT,
    warehouse=WAREHOUSE,
    database=DATABASE,
    schema='ZOOKEEPERS_BATCH_PRODUCTION'
)
cs = conn.cursor()
"""Already created schema to use"""

In [ ]:
query = "SHOW TABLES"
result = cs.execute(query)
result.fetchmany(100)

[]